## Tidal corrections from the ocean tide models EOT20 and FES2014

Exports a .csv-file with the corrections for a 10-minute-time vector from 01.01.1992 to 21.12.2021. The time vector is taken from the tide gauge observations for the North Sea tide gauge from rijkswaterstaat.

### Requirements to run this notebook
#### Installation
This notebook makes use of the [aviso-fes software](https://github.com/CNES/aviso-feshttps://github.com/CNES/aviso-fes). The python API `pyfes` is an Anaconda package, installation instructions can be found [here](https://anaconda.org/fbriol/pyfeshttps://anaconda.org/fbriol/pyfes).
#### Data
This notebook expects that the following datasets exist locally:
1. Rijkswaterstaat tide gauge observations with filename `'TG_rijkswaterstaat_with-999999999_1992_01_01-2021_12_31.csv''TG_rijkswaterstaat_with-999999999_1992_01_01-2021_12_31.csv'`. ***add how it is created***
2. EOT ocean tide model, downloadable [here](https://www.seanoe.org/data/00683/79489/https://www.seanoe.org/data/00683/79489/).
3. FES2014 (FES2014b ocean tides extrapolated, FES2014a load tides), downloadable on this FTP-server after signing up for an account: ftp-access.aviso.altimetry.frftp-access.aviso.altimetry.fr
more description on the FES2014 model is given [here](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes/https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes/).
4. Four configuration files for EOT/FES ocean tides/load tides, that tell the software which .nc-file belongs to which constituent. Configuration files for FES2014 can be downloaded from [github](https://github.com/CNES/aviso-fes/tree/main/data/fes2014https://github.com/CNES/aviso-fes/tree/main/data/fes2014). For use for EOT20 adapt the filenames to the .nc-files.

    EOT20 provides less constituents than FES2014. To be comparable, delete all the consitutents in FES2014, that are not apparent in EOT20 (in the filenames indicated as 'red-to-eot').

    Also, do not use SA and SSA as they include other signals as well and would remove the seasonal signal (in the filenames indicated as 'wo-SA-SSA').
### Ouput
Saves a .csv-file with tidal corrections for both models in the full 10-minute resolution (\~100MB), as well as monthly averages (\~23kB).


In [1]:
import pandas as pd
import pyfes
import numpy as np

Get the vector of dates from the tide gauge observation (10-minute vector from rijkswaterstaat North Sea tide gauge). Not every date has an observation, that's why the "with-999999999999999999"-version is loaded, but the ssh values are not of interest here.

In [2]:
# datapaths to the tide models
path_eot = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/2_EOT20/'
path_fes = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/1_FES2014/'

# datapaths to the configuration files
path_eot_ocean_tide = path_eot + 'ocean_tide_wo-SA-SSA.ini'
path_eot_load_tide = path_eot + 'load_tide_wo-SA-SSA.ini'
path_fes_ocean_tide = path_fes + 'ocean_tide_extrapolated_red-to-eot_wo-SA-SSA.ini'
path_fes_load_tide = path_fes + 'load_tide_red-to-eot_wo-SA-SSA.ini'

# savepath, where to put the .csv-file with the corrections
savepath = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/'

In [3]:
datapath_TG = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/6_tide_gauge/\
2_Ters-North_Sea_rijkswaterstaat/'
filename = 'TG_rijkswaterstaat_with-999999999_1992_01_01-2021_12_31.csv'
data = pd.read_csv(datapath_TG + filename, index_col='datetime[utc]', \
                  parse_dates=['datetime[utc]'])

date = data.index.values.astype('datetime64[us]')
# coordinates of the North-Sea-Tide-Gauge
lat = np.full(len(date), 53.420034)
lon = np.full(len(date), 5.351231)

Checks for duplicates, NaNs and gaps in the date vector:

In [4]:
print("Duplicates: ", data.index.duplicated().sum())
print("NaNs: ", data.isnull().sum())

diffs = np.diff(date)
idx = np.where(diffs != diffs[0])
len(idx[0])

Duplicates:  0
NaNs:  ssh[cm]    0
dtype: int64


0

In [5]:
ocean_tide_eot_handler = pyfes.Handler('ocean', 'io', path_eot_ocean_tide)
ocean_tide_eot = ocean_tide_eot_handler.calculate(lon, lat, date)

load_tide_eot_handler = pyfes.Handler('radial', 'io', path_eot_load_tide)
load_tide_eot = load_tide_eot_handler.calculate(lon, lat, date)

In [6]:
ocean_tide_fes_handler = pyfes.Handler('ocean', 'io', path_fes_ocean_tide)
ocean_tide_fes = ocean_tide_fes_handler.calculate(lon, lat, date)

load_tide_fes_handler = pyfes.Handler('radial', 'io', path_fes_load_tide)
load_tide_fes = load_tide_fes_handler.calculate(lon, lat, date)

In [7]:
# total correction (diurnal and semidiurnal + long period wave constituents for ocean and load tides)
corr_eot = ocean_tide_eot[0] + ocean_tide_eot[1] + load_tide_eot[0] + load_tide_eot[1]
corr_fes = ocean_tide_fes[0] + ocean_tide_fes[1] + load_tide_fes[0] + load_tide_fes[1]

In [8]:
# put date and both corrections in one dataframe
corr_df = pd.DataFrame(columns=['date', 'corr_eot[cm]', 'corr_fes[cm]'])
corr_df = corr_df.assign(date=data.index)
corr_df = corr_df.set_index(['date'])
corr_df['corr_eot[cm]'] = corr_eot
corr_df['corr_fes[cm]'] = corr_fes

# monthly averages
corr_df_monav = corr_df.groupby(pd.Grouper(freq='M')).mean()

In [9]:
savepath = '/media/bene/05748279-6089-4b9a-9546-c86b946cb93e/PhD-data/3_ocean_tide_models/'
corr_df.to_csv(savepath + "tidal_correction_10minutes.csv")
corr_df_monav.to_csv(savepath + "tidal_correction_monthly.csv")